<a href="https://colab.research.google.com/github/gouthamkallempudi/googletechdevguide/blob/master/Lesson3_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [0]:
X  = tf.placeholder(tf.float32 , [None , 784])
Y_ = tf.placeholder(tf.float32 , [None , 10])

In [0]:
W1 = tf.Variable(tf.truncated_normal([784 , 200] , stddev = 0.01))
b1 = tf.Variable(tf.ones([200])/100)

W2 = tf.Variable(tf.truncated_normal([200 , 100] , stddev = 0.01))
b2 = tf.Variable(tf.ones([100])/100)

W3 = tf.Variable(tf.truncated_normal([100 , 60] , stddev = 0.01))
b3 = tf.Variable(tf.ones([60])/100)

W4 = tf.Variable(tf.truncated_normal([60 , 30] , stddev = 0.01))
b4 = tf.Variable(tf.ones([30])/100)

W5 = tf.Variable(tf.truncated_normal([30 , 10] , stddev = 0.01))
b5 = tf.Variable(tf.ones([10])/100)

step = tf.placeholder(tf.int32)
pkeep = tf.placeholder(tf.float32)

In [0]:
def model_sigmoid():
  layer1 = tf.nn.sigmoid(tf.matmul(X , W1) + b1)
  layer1d = tf.nn.dropout(layer1 , pkeep)
  
  layer2 = tf.nn.sigmoid(tf.matmul(layer1 , W2) + b2)
  layer3 = tf.nn.sigmoid(tf.matmul(layer2 , W3) + b3)
  layer4 = tf.nn.sigmoid(tf.matmul(layer3 , W4) + b4)
  layer5 = tf.matmul(layer4 , W5) + b5
  
  Y = tf.nn.softmax(layer5)
  return Y , layer5
  
  

In [0]:
def model_relu():
  layer1 = tf.nn.relu(tf.matmul(X , W1) + b1)
  layer1d = tf.nn.dropout(layer1 , pkeep)
  
  layer2 = tf.nn.relu(tf.matmul(layer1d , W2) + b2)
  layer2d = tf.nn.dropout(layer2 , pkeep)
  
  layer3 = tf.nn.relu(tf.matmul(layer2d , W3) + b3)
  layer3d = tf.nn.dropout(layer3 , pkeep)
  
  layer4 = tf.nn.relu(tf.matmul(layer3d , W4) + b4)
  layer4d = tf.nn.dropout(layer4 , pkeep)
  
  layer5 = tf.matmul(layer4d , W5) + b5
  
  Y = tf.nn.softmax(layer5)
  return Y , layer5

In [0]:
Ylogits , Y = model_relu()
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100

is_correct = tf.equal(tf.argmax(Y , 1) , tf.argmax(Y_ , 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [0]:
import math
lr = 0.0001 + tf.train.exponential_decay(0.003, step, 2000, 1/math.e)
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)

In [95]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [101]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
  batch_X , batch_Y = mnist.train.next_batch(100)
  train_data = {X : batch_X , Y_ : batch_Y , pkeep : 0.7 , step: i}
  
  sess.run(train_step , feed_dict = train_data)
 
  if i%100 == 0:
    test_data={X: mnist.test.images, Y_: mnist.test.labels , pkeep : 1.0}
    a,c = sess.run([accuracy, cross_entropy], feed_dict=test_data)
    print(a , c)
   

0.1032 230.25882
0.3957 204.26068
0.6305 185.30276
0.7148 175.30746
0.792 167.52505
0.8722 159.11067
0.8929 156.81717
0.9063 155.4399
0.9099 155.15678
0.91 155.13837


In [97]:
print(a)
print(c)

0.9049
156.10382
